In [ ]:
import requests
import sqlite3
from datetime import datetime, timedelta
import logging
import json
from bs4 import BeautifulSoup
import pandas as pd
import openai
from dotenv import load_dotenv
import os

# Set your OpenAI API key
load_dotenv()
client = openai.OpenAI(api_key=os.getenv('API_KEY'))

def setup_logger(name):
    """Set up a logger for a given module."""
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    # Create file handler which logs even debug messages
    fh = logging.FileHandler('app.log')
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] - %(message)s')
    fh.setFormatter(formatter)

    # Add the handler to the logger
    if not logger.handlers:
        logger.addHandler(fh)

    return logger

logger = setup_logger(__name__)

def insert_news(df):
    conn = sqlite3.connect('../data-log.db')
    cursor = conn.cursor()

    # Function to check if a row with the same title exists and if relevant is not None
    def row_exists_and_relevant_not_none(title):
        cursor.execute("SELECT relevant FROM espn_news WHERE title = ?", (title,))
        row = cursor.fetchone()
        return row is not None and row[0] is not None

    for ix, row in df.iterrows():
        if row_exists_and_relevant_not_none(row['title']):
            # Update the row
            cursor.execute('''
            UPDATE espn_news
            SET date = ?, link = ?, image_url = ?, relevant = ?, ai_score = ?
            WHERE title = ?
            ''', (row['date'], row['link'], row['image_url'], row['relevant'], row['ai_score'], row['title']))
        else:
            # Insert the row if it doesn't exist
            cursor.execute('''
            INSERT OR IGNORE INTO espn_news (title, date, link, image_url, relevant, ai_score)
            VALUES (?, ?, ?, ?, ?, ?)
            ''', (row['title'], row['date'], row['link'], row['image_url'], row['relevant'], row['ai_score']))

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()

    logger.info("INSERTED NEWS TO espn_news")


def get_espn_news():
    # URL of the page to scrape
    url = "https://www.nfl.com/news/all-news"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # List to store the scraped data
        scraped_data = []

        # Function to extract article data
        def extract_article_data(article):
            # Extract the title
            title = article.find('h3', class_='d3-o-media-object__title').get_text(strip=True)

            # Extract the date
            date = article.find('p', class_='d3-o-media-object__date').get_text(strip=True)

            # Extract the link to the article
            link = article['href']
            full_link = f"https://www.nfl.com{link}"

            # Extract the image URL
            image_tag = article.find('picture').find('img')
            image_url = image_tag['src'] if image_tag else None

            # Append the extracted data to the list
            scraped_data.append({
                'title': title,
                'date': date,
                'link': full_link,
                'image_url': image_url
            })

        # Find all vertical article containers
        vertical_articles = soup.find_all('div', class_='d3-o-media-object--vertical')
        for article in vertical_articles:
            extract_article_data(article.find('a'))

        # Find all horizontal article containers
        horizontal_articles = soup.find_all('a', class_='d3-o-media-object--horizontal')
        for article in horizontal_articles:
            extract_article_data(article)

        df = pd.DataFrame(scraped_data)
        df['relevant'] = None
        df['ai_score'] = None
        insert_news(df)
        return df
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

def get_unclassified(start_date, end_date):
    # Connect to SQLite database
    conn = sqlite3.connect('../data-log.db')
    cursor = conn.cursor()

    # Execute the query to get articles within the date range where relevant is None
    cursor.execute('''
    SELECT * FROM espn_news
    WHERE relevant IS NULL
    ''')
    # 1
    # Fetch all results
    articles = cursor.fetchall()

    # Close the connection
    conn.close()

    columns = ['title', 'date', 'link', 'image_url', 'relevant', 'ai_score']
    df = pd.DataFrame(articles, columns=columns)
    #filter on_days
    df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))
    df = df[(df['date']>=start_date) & (df['date']<=end_date)]
    return df

def update_column(row_name, df):
    # Connect to SQLite database
    conn = sqlite3.connect('../data-log.db')
    cursor = conn.cursor()

    for ix, row in df.iterrows():
        cursor.execute(f'''
            UPDATE espn_news
            SET {row_name} = ?
            WHERE title = ?
            ''', (row[row_name], row['title']))
        
    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()
    logger.info(f"UPDATED {row_name} in espn_news")

def extract_article_text(article_url):
    article_response = requests.get(article_url)
    if article_response.status_code == 200:
        article_soup = BeautifulSoup(article_response.content, 'html.parser')
        article_body = article_soup.find('div', class_='nfl-c-article__body')
        if article_body:
            paragraphs = article_body.find_all('p')
            article_text = "\n".join([p.get_text(strip=True) for p in paragraphs])
            return article_text
    return None

def check_relevance(title):

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Reply True or False to whether the following news title is relevant to the odds of a team winning or losing'"},
        {"role": "user", "content": f"{title}"}
    ]
    )

    return completion.choices[0].message.content

def score_article(text):

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an assistant that reads article texts about sports teams. Your task is to identify the team name mentioned in the article and provide a rating on how the information in the article will affect that team's upcoming game. The rating should be on a scale from 5 to -5, where 5 indicates the team is sure to win and -5 indicates the team is sure to lose."},
        {"role": "system", "content": "Example return: [{'Tampa Bay Buccaneers': 3}]"},
        {"role": "system", "content": "always return json with a list of teams and impacts"},
        {"role": "system", "content": "always reply in the following format: [{'Team': Score}]"},
        {"role": "system", "content": "If data is missing or not relevant return [{'None': Score}]"},
        {"role": "system", "content": "dictionary values should be the rating score, keys should be in the following list Baltimore Ravens, Buffalo Bills, Chicago Bears, Cincinnati Bengals, Dallas Cowboys, Denver Broncos, Detroit Lions, Green Bay Packers, Houston Texans, Indianapolis Colts, Jacksonville Jaguars, Kansas City Chiefs, Los Angeles Chargers, Miami Dolphins, Minnesota Vikings, New Orleans Saints, New York Giants, New York Jets, Philadelphia Eagles, San Francisco 49ers, Seattle Seahawks, Tampa Bay Buccaneers, Washington Commanders"}
        {"role": "user", "content": f"{text}"}
    ],
     response_format={ "type": "json_object" }
    )

    return completion.choices[0].message.content

def insert_espn_news(start_date, end_date):
    logger.info("Starting espn_news updates")
    get_espn_news()
    df = get_unclassified(start_date, end_date)
    df["relevant"] = df["title"].apply(check_relevance)
    logger.info(f"RAN check_relevance on {len(df)} rows")
    update_column('relevant', df)
    #update_rows(df)
    df = df[df['relevant']=='True']
    df['article_text'] = df['link'].apply(extract_article_text)
    df['ai_score'] = df['article_text'].apply(score_article)
    logger.info(f"RAN score_article on {len(df)} rows")
    update_column('ai_score', df)
    logger.info("Completed espn_news updates")

In [95]:
', '.join(['Baltimore Ravens',
'Buffalo Bills',
'Chicago Bears',
'Cincinnati Bengals',
'Dallas Cowboys',
'Denver Broncos',
'Detroit Lions',
'Green Bay Packers',
'Houston Texans',
'Indianapolis Colts',
'Jacksonville Jaguars',
'Kansas City Chiefs',
'Los Angeles Chargers',
'Miami Dolphins',
'Minnesota Vikings',
'New Orleans Saints',
'New York Giants',
'New York Jets',
'Philadelphia Eagles',
'San Francisco 49ers',
'Seattle Seahawks',
'Tampa Bay Buccaneers',
'Washington Commanders'])

'Baltimore Ravens, Buffalo Bills, Chicago Bears, Cincinnati Bengals, Dallas Cowboys, Denver Broncos, Detroit Lions, Green Bay Packers, Houston Texans, Indianapolis Colts, Jacksonville Jaguars, Kansas City Chiefs, Los Angeles Chargers, Miami Dolphins, Minnesota Vikings, New Orleans Saints, New York Giants, New York Jets, Philadelphia Eagles, San Francisco 49ers, Seattle Seahawks, Tampa Bay Buccaneers, Washington Commanders'

In [42]:
with sqlite3.connect('data-log.db') as conn:
        cursor = conn.cursor()

        # Fetch the corresponding game data from the database by game_id
        cursor.execute("""
                    SELECT * FROM espn_news 
                    """)
columns = ['title', 'date', 'link', 'image_url', 'relevant', 'ai_score']
df = pd.DataFrame(cursor.fetchall(), columns=columns)

In [43]:
def get_start_end():
    today = datetime.now()
    weekday = today.weekday()  # Monday is 0 and Sunday is 6

    # Calculate the start date (Tuesday)
    if weekday >= 1:  # If today is Tuesday or after
        start_date = today - timedelta(days=(weekday - 1))
    else:  # If today is before Tuesday
        start_date = today - timedelta(days=(weekday + 6))

    # Calculate the end date (Monday)
    if weekday <= 0:  # If today is Monday
        end_date = today
    else:  # If today is after Monday
        end_date = today + timedelta(days=(7 - weekday))
    return start_date, end_date

start_date, end_date = get_start_end()

# Connect to SQLite database
with sqlite3.connect('data-log.db') as conn:
    cursor = conn.cursor()

    # Execute the query to get articles within the date range where relevant is None
    cursor.execute('''
    SELECT * FROM espn_news
    ''')
# 1
# Fetch all results
data = cursor.fetchall()

columns = ['title', 'date', 'link', 'image_url', 'relevant', 'ai_score']
df = pd.DataFrame(data, columns=columns)

# df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))
# df = df[(df['date']>=start_date) & (df['date']<=end_date)]

final_results = []

df = df[df['ai_score'].notna()]
df['ai_score'] = df['ai_score'].str.replace('\n', '')
# def get_key(x):
#     try:
#         load = json.loads(x)
#         key =  list(load.keys())[0]
#         if key.lower() in ['result', 'results', 'team', 'team_name']:
#             return load[key]
#         return key
#     except:
#         return None

# def get_value(x):
#     try:
#         return list(json.loads(x).values())[0]
#     except:
#         return None
    
# """
# LOOP!
# If len keys >1 create new row!
# """

# df['team'] = df['ai_score'].apply(get_key)
# df['impact'] = df['ai_score'].apply(get_value)

In [91]:
items = []
for ix, row in df.iterrows():
    load = json.loads(row['ai_score'])
    keys =  list(load.keys())
    # if key.lower() in ['result', 'results', 'team', 'team_name']:
    #     return load[key]
    if len(keys)!=0:
        if keys[0].lower() in ['result', 'results', 'team', 'team_name']:
            if len(load[keys[0]]) > 1:
                try:
                    for r in load[keys[0]]:
                        item = {
                            "title": row['title'],
                            "team": list(r.keys())[0],
                            "rating": list(r.values())[0]
                        }
                        items.append(item)
                except:
                    item = {
                        'title': row['title'],
                        "team": load[keys[0]],
                        "rating": load[keys[1]]
                        }
                    items.append(item)
            else:
                item = {
                    "title": row['title'],
                    "team": list(load[keys[0]].keys())[0],
                    "rating": load[keys[0]][list(load[keys[0]].keys())[0]]
                }
                items.append(item)
        else:
            item = {
                    "title": row['title'],
                    "team": keys[0],
                    "rating": load[keys[0]]
                }
            items.append(item)


In [92]:
items

[{'title': 'Bears fire Matt Eberflus following 4-8 start to season; Thomas Brown named interim head coach',
  'team': 'Chicago Bears',
  'rating': -3},
 {'title': "Jaguars' Doug Pederson: Trevor Lawrence (shoulder) expected to play Sunday vs. Texans 'as of right now'",
  'team': 'Jacksonville Jaguars',
  'rating': -2},
 {'title': 'NFL Week 13 bold predictions: T.J. Hockenson posts hat trick; Ladd McConkey and C.J. Stroud ball out',
  'team': 'T.J. Hockenson',
  'rating': 1},
 {'title': 'NFL Week 13 bold predictions: T.J. Hockenson posts hat trick; Ladd McConkey and C.J. Stroud ball out',
  'team': 'Los Angeles Chargers',
  'rating': 3},
 {'title': 'NFL Week 13 bold predictions: T.J. Hockenson posts hat trick; Ladd McConkey and C.J. Stroud ball out',
  'team': 'Houston Texans',
  'rating': 4},
 {'title': 'NFL Week 13 bold predictions: T.J. Hockenson posts hat trick; Ladd McConkey and C.J. Stroud ball out',
  'team': 'Philadelphia Eagles',
  'rating': 0},
 {'title': 'NFL Week 13 bold pre

In [93]:
pd.DataFrame(items)

,title,team,rating
0,Bears fire Matt Eberflus following 4-8 start t...,Chicago Bears,-3
1,Jaguars' Doug Pederson: Trevor Lawrence (shoul...,Jacksonville Jaguars,-2
2,NFL Week 13 bold predictions: T.J. Hockenson p...,T.J. Hockenson,1
3,NFL Week 13 bold predictions: T.J. Hockenson p...,Los Angeles Chargers,3
4,NFL Week 13 bold predictions: T.J. Hockenson p...,Houston Texans,4
5,NFL Week 13 bold predictions: T.J. Hockenson p...,Philadelphia Eagles,0
6,NFL Week 13 bold predictions: T.J. Hockenson p...,Baltimore Ravens,0
7,NFL Week 13 bold predictions: T.J. Hockenson p...,New Orleans Saints,2
8,Giants HC Brian Daboll not concerned with job ...,New York Giants,-5
9,Cowboys outlast Giants for first home win of s...,Dallas Cowboys,2


In [70]:

keys

['team_name', 'rating']

In [46]:
df

,title,date,link,image_url,relevant,ai_score
1,Bears fire Matt Eberflus following 4-8 start t...,"Nov 29, 2024",https://www.nfl.com/news/bears-firing-head-coa...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,...
3,Jaguars' Doug Pederson: Trevor Lawrence (shoul...,"Nov 29, 2024",https://www.nfl.com/news/jaguars-doug-pederson...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Jacksonville Jaguars"": -2}"
5,NFL Week 13 bold predictions: T.J. Hockenson p...,"Nov 29, 2024",https://www.nfl.com/news/nfl-week-13-bold-pred...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""results"": [ {""T.J. Hockenson"": 1}, {..."
8,Giants HC Brian Daboll not concerned with job ...,"Nov 29, 2024",https://www.nfl.com/news/brian-daboll-not-conc...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""New York Giants"": -5}"
10,Cowboys outlast Giants for first home win of s...,"Nov 28, 2024",https://www.nfl.com/news/cowboys-outlast-giant...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Dallas Cowboys"": 2}"
12,Bears HC Matt Eberflus defends not calling tim...,"Nov 28, 2024",https://www.nfl.com/news/bears-matt-eberflus-c...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""team"": { ""Chicago Bears"": -3 } }"
14,Dolphins won't activate Shaquil Barrett off re...,"Nov 28, 2024",https://www.nfl.com/news/dolphins-won-t-activa...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""Miami Dolphins"": -1 }"
15,Drew Lock to start for Giants on Thanksgiving ...,"Nov 28, 2024",https://www.nfl.com/news/drew-lock-to-start-fo...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""New York Giants"": -4}"
16,Chiefs activate Isiah Pacheco (fibula) off inj...,"Nov 28, 2024",https://www.nfl.com/news/chiefs-activate-isiah...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"\t{""Kansas City Chiefs"": 4}"
18,"Lions' David Montgomery, Amon-Ra St. Brown act...","Nov 28, 2024",https://www.nfl.com/news/lions-david-montgomer...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Detroit Lions"": 4}"


In [45]:
for r in load[keys[0]]:
    print(list(r.keys())[0])

T.J. Hockenson
Los Angeles Chargers
Houston Texans
Philadelphia Eagles
Baltimore Ravens
New Orleans Saints


In [27]:
len(keys)

0

In [95]:
json.loads('{"Jacksonville Jaguars": -2}').keys()

dict_keys(['Jacksonville Jaguars'])

In [86]:
df

,title,date,link,image_url,relevant,ai_score
1,Bears fire Matt Eberflus following 4-8 start t...,2024-11-29,https://www.nfl.com/news/bears-firing-head-coa...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,...
3,Jaguars' Doug Pederson: Trevor Lawrence (shoul...,2024-11-29,https://www.nfl.com/news/jaguars-doug-pederson...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Jacksonville Jaguars"": -2}"
5,NFL Week 13 bold predictions: T.J. Hockenson p...,2024-11-29,https://www.nfl.com/news/nfl-week-13-bold-pred...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""results"": [ {""T.J. Hockenson"": 1}, {..."
8,Giants HC Brian Daboll not concerned with job ...,2024-11-29,https://www.nfl.com/news/brian-daboll-not-conc...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""New York Giants"": -5}"
10,Cowboys outlast Giants for first home win of s...,2024-11-28,https://www.nfl.com/news/cowboys-outlast-giant...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Dallas Cowboys"": 2}"
12,Bears HC Matt Eberflus defends not calling tim...,2024-11-28,https://www.nfl.com/news/bears-matt-eberflus-c...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""team"": { ""Chicago Bears"": -3 } }"
14,Dolphins won't activate Shaquil Barrett off re...,2024-11-28,https://www.nfl.com/news/dolphins-won-t-activa...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{ ""Miami Dolphins"": -1 }"
15,Drew Lock to start for Giants on Thanksgiving ...,2024-11-28,https://www.nfl.com/news/drew-lock-to-start-fo...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""New York Giants"": -4}"
16,Chiefs activate Isiah Pacheco (fibula) off inj...,2024-11-28,https://www.nfl.com/news/chiefs-activate-isiah...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"\t{""Kansas City Chiefs"": 4}"
18,"Lions' David Montgomery, Amon-Ra St. Brown act...",2024-11-28,https://www.nfl.com/news/lions-david-montgomer...,"data:image/gif;base64,R0lGODlhAQABAIAAAP///wAA...",True,"{""Detroit Lions"": 4}"


In [ ]:

for ix, row in df.iterrows():
    ai_score = json.loads(row['ai_score'].strip().replace('\n', ''))
    df_keys = list(ai_score.keys())
    #df.at[ix, 'ai_score'] = 
    if df_keys:
        if 'result' in ai_score or 'results' in ai_score:
            for game in ai_score[df_keys[0]]:
                final_results.append(game)
        else:
            final_results.append(row['ai_score'])

def cast_int_or_zero(value):
    try:
        return(int(value))
    except:
        return None
    
df = pd.DataFrame(final_results)
df[0] = df[0].apply(lambda x: x.replace('\n', '').strip() if type(x) == str else x)
df['json'] = df[0].apply(lambda x: json.dumps(x))

In [71]:
df[0]

0                               {  "Chicago Bears": -3}
1                          {"Jacksonville Jaguars": -2}
2                                 {'T.J. Hockenson': 1}
3                           {'Los Angeles Chargers': 3}
4                                 {'Houston Texans': 4}
5                            {'Philadelphia Eagles': 0}
6                               {'Baltimore Ravens': 0}
7                             {'New Orleans Saints': 2}
8                               {"New York Giants": -5}
9                                 {"Dallas Cowboys": 2}
10    {    "team": {        "Chicago Bears": -3    }  }
11                             { "Miami Dolphins": -1 }
12                              {"New York Giants": -4}
13                            {"Kansas City Chiefs": 4}
14                                 {"Detroit Lions": 4}
15                          {"Tampa Bay Buccaneers": 3}
16                          {"San Francisco 49ers": -2}
17                        {"Washington Commander

In [72]:
# Function to parse JSON-like strings
def parse_json_like_string(s):
    try:
        # Replace single quotes with double quotes
        s = s.replace("'", '"')
        # Remove leading and trailing spaces
        s = s.strip()
        # Parse the JSON string
        return json.loads(s)
    except json.JSONDecodeError:
        return None

# Parse the JSON-like strings
df['parsed'] = df[0].apply(parse_json_like_string)

# Normalize the parsed data
normalized_df = pd.json_normalize(df['parsed'])

AttributeError: 'dict' object has no attribute 'replace'

In [ ]:
for i, x in df.iterrows():
    try:
        cleaned = json.loads(x[0].replace(' ', ''))
    except:
        print("didnt work")
        cleaned = x[0]
        break
    print(cleaned)#.keys()
    #print(cleaned).values()


{'ChicagoBears': -3}
{'JacksonvilleJaguars': -2}
didnt work


In [69]:
cleaned.keys()

dict_keys(['T.J. Hockenson'])

In [ ]:


# Melt the DataFrame to long format
df_melted = df.melt(var_name='TEAM')
# Drop rows with NaN values in the Score column
df_melted['value'] = df_melted['value'].apply(lambda x: cast_int_or_zero(x))
df_melted = df_melted.dropna(subset=['value'])

# Group by TEAM and calculate AVG and SUM
df_grouped = df_melted.groupby('TEAM')['value'].agg(['mean', 'sum']).reset_index()

# Rename columns
df_grouped.columns = ['TEAM', 'AVG', 'SUM']

teams = ['Arizona Cardinals',
'Baltimore Ravens',
'Buffalo Bills',
'Chicago Bears',
'Cincinnati Bengals',
'Dallas Cowboys',
'Denver Broncos',
'Detroit Lions',
'Green Bay Packers',
'Houston Texans',
'Indianapolis Colts',
'Jacksonville Jaguars',
'Kansas City Chiefs',
'Los Angeles Chargers',
'Miami Dolphins',
'Minnesota Vikings',
'New Orleans Saints',
'New York Giants',
'New York Jets',
'Philadelphia Eagles',
'San Francisco 49ers',
'Seattle Seahawks',
'Tampa Bay Buccaneers',
'Washington Commanders',]

In [6]:
df_grouped[df_grouped['TEAM'].isin(teams)]

,TEAM,AVG,SUM
